<a href="https://colab.research.google.com/github/glutamatt/semi_perso_user_cold_start/blob/colab/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!rm -fr repository deezer data
!git clone https://github.com/glutamatt/semi_perso_user_cold_start.git repository
!cd repository && git checkout colab && cd -

Cloning into 'repository'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 113 (delta 60), reused 110 (delta 57), pack-reused 0
Receiving objects: 100% (113/113), 585.07 KiB | 11.25 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Branch 'colab' set up to track remote branch 'colab' from 'origin'.
Switched to a new branch 'colab'
/content


# Test de contenu

impec 

## Installation

### Code

```Bash
git clone https://github.com/deezer/carousel_bandits
cd carousel_bandits
```

Requirements: python 3, matplotlib, numpy, pandas, scipy, seaborn

Media services providers, such as the music streaming platform [Deezer](https://www.deezer.com/), often leverage **swipeable carousels** to recommend personalized content to their users. These carousels are ranked lists of _L_ items or **cards** from a substantially larger catalog (of size _K_), e.g. _L_ albums, artists or playlists recommended on the homepage of the Deezer app. Only a few cards, say _L_init_ < _L_, are initially displayed to users, who can **swipe** the screen to see additional cards.

Selecting the most relevant content to display in carousels is a challenging task, as the catalog is large and as users have different preferences. Also, ranking matters: some cards might not be seen by some users due to the swipeable structure.

<p align="center">
  <img height="350" src="https://raw.githubusercontent.com/deezer/carousel_bandits/master/images/carousel.png">
</p>

In [8]:
!pip install fastparquet

In [9]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn
import time
import statistics
import pickle
import random
from sklearn.metrics import ndcg_score, dcg_score
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer

In [17]:
def generate(master_path):
    dataset_path = os.getcwd() + "/repository/data"

    #songs

    song_embeddings_path = dataset_path + "/song_embeddings.parquet"
    song_embeddings = pd.read_parquet(song_embeddings_path, engine = 'fastparquet').fillna(0)

    if not os.path.exists(master_path+"/m_song_dict.pkl"):
        song_dict = {}
        for idx, row in song_embeddings.iterrows():
            song_dict[row['song_index']] = idx
        pickle.dump(song_dict, open("{}/m_song_dict.pkl".format(master_path), "wb"))
    else:
        song_dict = pickle.load(open("{}/m_song_dict.pkl".format(master_path), "rb"))


    # user embeddings (target = only for train users)

    user_embeddings = pd.read_parquet(dataset_path + "/user_embeddings.parquet", engine = 'fastparquet')
    list_embeddings = ["embedding_"+str(i) for i in range(len(user_embeddings["svd_embeddings"][0]))]
    user_embeddings[list_embeddings] = pd.DataFrame(user_embeddings.svd_embeddings.tolist(), index= user_embeddings.index)
    embeddings_train = user_embeddings[list_embeddings].values

    # user features train

    features_train_path = dataset_path + "/user_features_train.parquet"
    features_train = pd.read_parquet(features_train_path, engine = 'fastparquet').fillna(0)
    features_train = features_train.sort_values("user_index")
    features_train = features_train.reset_index(drop=True)#to check it is ok for train data

    # training dataset creation

    state = "train"
    if not os.path.exists(master_path+"/"):
        os.mkdir(master_path+"/")
    if not os.path.exists(master_path+"/"+state+"/"):
        os.mkdir(master_path+"/"+state+"/")
    for idx in range(len(features_train)):
        x_train = torch.FloatTensor(features_train.iloc[idx,2:])
        y_train = torch.FloatTensor(user_embeddings[list_embeddings].iloc[idx,:])
        pickle.dump(x_train, open("{}/{}/x_train_{}.pkl".format(master_path, state, idx), "wb"))
        pickle.dump(y_train, open("{}/{}/y_train_{}.pkl".format(master_path, state, idx), "wb"))

    # user features validation & test

    states = ["validation", "test"]
    for state in states :
        features_validation_path = dataset_path + "/user_features_" + state + ".parquet"
        features_validation = pd.read_parquet(features_validation_path, engine = 'fastparquet').fillna(0)
        features_validation = features_validation.sort_values("user_index")
        features_validation = features_validation.reset_index(drop=True)

        if not os.path.exists(master_path+"/"+state+"/"):
            os.mkdir(master_path+"/"+state+"/"+"/")
        for i in range(len(features_validation)):
            x_validation = torch.FloatTensor(features_validation.iloc[i,2:])
            y_validation = [song_dict[song_index]  for song_index in features_validation["d1d30_songs"][i]]
            groundtruth_validation_list = [1.0 * (song in y_validation) for song in range(len(song_embeddings))]
            pickle.dump(x_validation, open("{}/{}/x_validation_{}.pkl".format(master_path, state, i), "wb"))
            pickle.dump(y_validation, open("{}/{}/y_listened_songs_validation_{}.pkl".format(master_path, state, i), "wb"))
            pickle.dump(groundtruth_validation_list, open("{}/{}/groundtruth_list_{}.pkl".format(master_path, state, i), "wb"))


In [20]:
master_path= "./deezer"
if not os.path.exists("{}/".format(master_path)):
  os.mkdir("{}/".format(master_path))
  # preparing dataset. It needs about XXGB of your hard disk space.
  generate(master_path)
else:
  print("%s already exists" % master_path)

./deezer already exists


In [25]:
!apt install -qqy tree && tree deezer

tree is already the newest version (1.7.0-5).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
deezer
├── m_song_dict.pkl
├── test
│   ├── groundtruth_list_0.pkl
│   ├── groundtruth_list_1.pkl
│   ├── groundtruth_list_2.pkl
│   ├── groundtruth_list_3.pkl
│   ├── groundtruth_list_4.pkl
│   ├── x_validation_0.pkl
│   ├── x_validation_1.pkl
│   ├── x_validation_2.pkl
│   ├── x_validation_3.pkl
│   ├── x_validation_4.pkl
│   ├── y_listened_songs_validation_0.pkl
│   ├── y_listened_songs_validation_1.pkl
│   ├── y_listened_songs_validation_2.pkl
│   ├── y_listened_songs_validation_3.pkl
│   └── y_listened_songs_validation_4.pkl
├── train
│   ├── x_train_0.pkl
│   ├── x_train_1.pkl
│   ├── x_train_2.pkl
│   ├── x_train_3.pkl
│   ├── x_train_4.pkl
│   ├── y_train_0.pkl
│   ├── y_train_1.pkl
│   ├── y_train_2.pkl
│   ├── y_train_3.pkl
│   └── y_train_4.pkl
└── validation
    ├── groundtruth_list_0.pkl
    ├── groundtruth_list_1.pkl
    ├── groundtruth_list_2.pkl
    ├── groundtr